In [70]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from requests_testadapter import Resp
import requests
#from requests_file import FileAdapter
import os.path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import string
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from nltk import word_tokenize, sent_tokenize
from sklearn.decomposition import NMF


In [71]:
df_reviews = pd.read_csv('reviews_sep2_clean.csv')

In [72]:
def valoracion_a_numeros(valor):
    if valor == 'Excelente':
        return 4
    if valor == 'Muy bueno':
        return 3
    if valor == 'Bueno':
        return 2
    if valor == 'Regular':
        return 1

columnas_valoracion = ['comida','servicio','ambiente']
for item in columnas_valoracion:
    df_reviews[item] = df_reviews[item].apply(valoracion_a_numeros)

print ('valores unicos en COMIDA',df_reviews.comida.unique())
print ('valores unicos en SERVICIO',df_reviews.servicio.unique())
print ('valores unicos en AMBIENTE',df_reviews.ambiente.unique())


valores unicos en COMIDA [4 3 2 1]
valores unicos en SERVICIO [4 1 3 2]
valores unicos en AMBIENTE [4 3 1 2]


In [73]:
mask_negative = (df_reviews["comida"] < 2) & (df_reviews["servicio"] < 2) & (df_reviews["ambiente"] < 2)
mask_positive = (df_reviews["comida"] > 3) & (df_reviews["servicio"] > 3 ) & (df_reviews["ambiente"] > 3)
# mask_com = df_reviews['comida'] = 1
# mask_serv = df_reviews['servicio'] = 1
# mask_amb = df_reviews['ambiente'] = 1
# df_reviews_neg = df_reviews[mask_com] & df_reviews[mask_serv] & df_reviews[mask_amb]

print ('reviews originales',df_reviews.fecha.count())

df_negative = df_reviews[mask_negative]
print ('reviews segmentadas negativas',df_negative.fecha.count())

df_positive = df_reviews[mask_positive]
print ('reviews segmentadas positivas',df_positive.fecha.count())

reviews originales 28286
reviews segmentadas negativas 1949
reviews segmentadas positivas 3796


In [74]:
vectorizer = TfidfVectorizer(max_df=1.0, max_features=100000,
                             min_df=0.01, use_idf=True) #le saque el stopwords, porque ya lo hicimos antes

corpus = vectorizer.fit_transform(df_negative.nopoint)

In [75]:
dim = 3
nmf = NMF(n_components = dim)
nmf_array = nmf.fit_transform(corpus)

In [76]:
# Chequeamos los shapes de las matrices resultantes
print('Matriz de documentos por tópicos:', nmf_array.shape)
print('Matriz de tópicos por términos:', nmf.components_.shape)

Matriz de documentos por tópicos: (1949, 3)
Matriz de tópicos por términos: (3, 482)


In [77]:
labels = [np.argmax(x) for x in nmf_array]

In [78]:
# Componentes y nombre de los feautures
components = [nmf.components_[i] for i in range(len(nmf.components_))]
features = vectorizer.get_feature_names()

for j in range(len(components)):

    comp = components[j]

    prior_features = sorted(features, key = lambda x: comp[features.index(x)], reverse = True)

    # Guardamos en archivos los features de cada componente ordenados por prioridad y 
    # las notas asociadas a cada tópico.
    fp = open('NMFComponentpositive{}.txt'.format(j), 'a')
    for k in prior_features:
        fp.write('{}, '.format(k))
    fp.close()
        
    fp = open('NMFNotes{}.txt'.format(j),'a')
    for k in range(len(labels)):
        if labels[k] == j:
            fp.write('{}, '.format(k))
            
    fp.close()

In [79]:
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(3):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topico negative # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [80]:
get_nmf_topics(nmf, 3)

,Topico negative # 01,Topico negative # 02,Topico negative # 03
0,mas,comida,lugar
1,pedimos,mala,si
2,nunca,atencion,comer
3,mesa,atención,ir
4,dos,calidad,pizza
5,trajeron,buena,bien
6,desastre,precios,mal
7,hora,recomiendo,gente
8,plato,ambiente,recomiendo
9,pedido,malo,buenos


In [81]:
corpus = vectorizer.fit_transform(df_negative.nopoint)
dim = 3
nmf = NMF(n_components = dim)
nmf_array = nmf.fit_transform(corpus)
labels = [np.argmax(x) for x in nmf_array]

In [82]:
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(3):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topico positive # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [83]:
# Componentes y nombre de los feautures
components = [nmf.components_[i] for i in range(len(nmf.components_))]
features = vectorizer.get_feature_names()

for j in range(len(components)):

    comp = components[j]

    prior_features = sorted(features, key = lambda x: comp[features.index(x)], reverse = True)

    # Guardamos en archivos los features de cada componente ordenados por prioridad y 
    # las notas asociadas a cada tópico.
    fp = open('NMFComponent{}.txt'.format(j), 'a')
    for k in prior_features:
        fp.write('{}, '.format(k))
    fp.close()
        
    fp = open('NMFNotes{}.txt'.format(j),'a')
    for k in range(len(labels)):
        if labels[k] == j:
            fp.write('{}, '.format(k))
            
    fp.close()

In [84]:
get_nmf_topics(nmf, 3)

,Topico positive # 01,Topico positive # 02,Topico positive # 03
0,mas,comida,lugar
1,pedimos,mala,si
2,nunca,atencion,comer
3,mesa,atención,ir
4,dos,calidad,pizza
5,trajeron,buena,bien
6,desastre,precios,mal
7,hora,recomiendo,gente
8,plato,ambiente,recomiendo
9,pedido,malo,buenos
